In [3]:
# Numerical tools
import numpy as np
import numpy as np
from scipy import signal
from sklearn.decomposition import PCA, KernelPCA, FastICA

# Viz
import matplotlib.pyplot as plt

# PCA, kernel PCA, and ICA

This notebook is inspired by an interest in unsupervised data exploration. It might use some concept from the notebook on information theory.
We also used this nice blog article: https://towardsdatascience.com/kernel-pca-vs-pca-vs-ica-in-tensorflow-sklearn-60e17eb15a64

In [1]:
from IPython.display import IFrame
IFrame("doc/KPCA.pdf", width=800, height=600)

### sklearn.decomposition.PCA
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
### sklearn.decomposition.KernelPCA
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA.html
### sklearn.decomposition.FastICA
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html


In [ ]:
# Example inspired by https://scikit-learn.org/stable/auto_examples/decomposition/plot_ica_blind_source_separation.html#sphx-glr-auto-examples-decomposition-plot-ica-blind-source-separation-py
# #############################################################################
# Generate sample data
np.random.seed(0)
n_samples = 2000
time = np.linspace(0, 8, n_samples)

s1 = np.sin(2 * time)  # Signal 1 : sinusoidal signal
s2 = np.sign(np.sin(3 * time))  # Signal 2 : square signal
s3 = signal.sawtooth(2 * np.pi * time)  # Signal 3: saw tooth signal

S = np.c_[s1, s2, s3]
S += 0.2 * np.random.normal(size=S.shape)  # Add noise

S /= S.std(axis=0)  # Standardize data
# Mix data
A = np.array([[1, 1, 1], [0.5, 2, 1.0], [1.5, 1.0, 2.0]])  # Mixing matrix
X = np.dot(S, A.T)  # Generate observations

# Compute ICA
ica = FastICA(n_components=3)
S_ = ica.fit_transform(X)  # Reconstruct signals
A_ = ica.mixing_  # Get estimated mixing matrix

# We can `prove` that the ICA model applies by reverting the unmixing.
assert np.allclose(X, np.dot(S_, A_.T) + ica.mean_)

# For comparison, compute PCA
pca = PCA(n_components=3)
H = pca.fit_transform(X)  # Reconstruct signals based on orthogonal components

# #############################################################################
# Plot results

plt.figure()

models = [X, S, S_, H]
names = ['Observations (mixed signal)',
         'True Sources',
         'ICA recovered signals',
         'PCA recovered signals']
colors = ['red', 'steelblue', 'orange']

for ii, (model, name) in enumerate(zip(models, names), 1):
    plt.subplot(4, 1, ii)
    plt.title(name)
    for sig, color in zip(model.T, colors):
        plt.plot(sig, color=color)

plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.46)
plt.show()